Importação de warnings

In [1]:
import warnings
#Controle de avisos
warnings.filterwarnings("ignore")

# Intalação da CrewAI

In [2]:
!python -m pip install crewai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.3/265.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 73.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 4.5 MB/s eta 0:0

# Importação de ulteriores bibliotecas necessárias

In [23]:
from crewai import Agent, Task, Crew
import os
from IPython.display import Markdown

#Definindo a chave de API do OpenAI
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
#Definindo o modelo do ChatGPT que será usado
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

# Criação dos agentes

In [24]:
#Criação do Agente planner que irá planejar o conteúdo
planner = Agent(
    #Definição de seu papel
    role="Content Planner",
    #Definição de seu objetivo
    goal="Plan engaging and factually acurate content on {topic}",
    #Definição de um certo contexto ao agente, para que ele comece a entender melhor seu papel
    backstory="You're working on planning a blog article about the topic: {topic}."
    "You collect information that helps the audience learn something and make informed decisions."
    "Your work is the basis for the Content Writer to write an article on this topic",
    allow_delegation=False,
    #Define a exibição da geração do agente
    verbose=True
)

In [25]:
#Criação do Agente writer que irá escrever o conteúdo
writer = Agent(
    #Definição de seu papel
    role="Content Writer",
    #Definição de seu objetivo
    goal="Write insightful and factually accurate opinion piece about the topic: {topic}",
    #Definição de um certo contexto ao agente, para que ele comece a entender melhor seu papel
    backstory="You're working on a writing a new opinion piece about the topic: {topic}."
    "You base your writing on the work of the Content Planner, who provides an outline and relevant context about the topic."
    "You follow the main objectives and direction of the outline, as provide by the Content Planner."
    "You also provide objective and impartial insights and back them up with information provide by the Content Planner."
    "You acknowledge in your opinion piece when your statements are opinions as opposed to objective statements.",
    allow_delegation=False,
    #Define a exibição da geração do agente
    verbose=True
)

In [26]:
#Criação do Agente editor que irá editar o conteúdo
editor = Agent(
    #Definição de seu papel
    role="Editor",
    #Definição de seu objetivo
    goal="Edit a given blog post to align with the writing style of the organization.",
    #Definição de um certo contexto ao agente, para que ele comece a entender melhor seu papel
    backstory="You are an editor who receives a blog post from the Content Writer."
    "Your goal is to review the blog post to ensure that it follows journalistic"
    "best practices, provides balanced viewpoints when providing opinions or assertions,"
    "and also avoids major controversial topics or opinions when possible.",
    allow_delegation=False,
    #Define a exibição da geração do agente
    verbose=True
)

# Criação das tarefas a serem executadas

In [27]:
#Criação da task de planejamento
plan = Task(
    #Descrição da task a ser realizada
    description=(
        "1. Prioritize the latest trends, key players, and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering their interests and pain points.\n"
        "3. Develop a detailed content outline including an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    #Saída esperada
    expected_output="A comprehensive content plan document with an outline, audience analysis, SEO keywords, and resources.",
    #Definição do Agetne responsável pela realização da task
    agent=planner
)

In [28]:
#Criação da task de escrita
write = Task(
    #Descrição da task a ser realizada
    description=(
        "1. Use the content plan to craft a compelling blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
        "3. Sections/Subtitles are properly named in an engaging manner.\n"
        "4. Ensure the post is structured with an engaging introduction, insightful body, and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and alignment with the brand's voice.\n"
    ),
    #Saída esperada
    expected_output="A well-written blog post in markdown format, ready for publication, each section should have 2 or 3 paragraphs.",
    #Definição do Agetne responsável pela realização da task
    agent=writer
)

In [29]:
#Criação da task de edição
edit = Task(
    #Descrição da task a ser realizada
    description=("Proofread the given blog post for grammatical errors and alignment with the brand's voice."),
    #Saída esperada
    expected_output="A well-written blog post in markdown format, ready for publication. each section should have 2 or 3 paragraphs.",
    #Definição do Agetne responsável pela realização da task
    agent=editor
)

# Criação do Crew(permite que os agetnes trabalhem em grupo)

In [30]:
crew = Crew(
    #Agentes que fazem parte da crew
    agents=[planner, writer, editor],
    #Definição das tasks na sequência a ser realizada pelo seu respectivo agente
    tasks=[plan, write, edit],
    #Define que a saída gerada será exibida
    verbose=True
)

# Rodando o crew e definindo o tópico a ser abordado

In [31]:
#Tópico definido: "Inteligência Artificial"
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 0640041f-4635-42a9-b5d8-6f94547a869f                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: a2ca9860-e4c2-49bf-8030-451ecab4ede1
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: a2ca9860-e4c2-49bf-8030-451ecab4ede1
       Status: Executing Task...
    └── 🤖 Agent: Content Planner
            Status: In Progress

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


🚀 Crew: crew
└── 📋 Task: a2ca9860-e4c2-49bf-8030-451ecab4ede1
       Status: Executing Task...
    └── 🤖 Agent: Content Planner
            Status: In Progress
        └── 🧠 Thinking...

...



# Agent: Editor
## Final Answer: 
# Unraveling Artificial Intelligence: A Comprehensive Guide to the Latest Trends and Key Insights

## Introduction
Artificial Intelligence (AI) is reshaping how we interact with technology, enabling machines to learn, adapt, and execute tasks that previously relied on human intelligence. This blog post delves deep into AI, covering its definition, evolution over time, and its pivotal role across various industries. From streamlining automation processes to enhancing decision-making capabilities, AI has become an essential component in today's technological landscape.

The Latest Trends in Artificial Intelligence
The realm of AI is experiencing rapid progress, particularly in machine learning algorithms that power intelligent systems capable of improving performance continually without explicit programming. Additionally, the rise of autonomous vehicles and robotics demonstrates how AI is revolutionizing transportation and manufacturing sectors. In hea

🚀 Crew: crew
├── 📋 Task: a2ca9860-e4c2-49bf-8030-451ecab4ede1
│      Assigned to: Content Planner
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Planner
│           Status: ✅ Completed
├── 📋 Task: 09be3ee7-1dd3-455f-a325-4f193ec8d026
│      Assigned to: Content Writer
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Writer
│           Status: ✅ Completed
└── 📋 Task: 3d3a4878-13b5-47d6-99c6-b4aa7a1a4ad0
       Status: Executing Task...
    └── 🤖 Agent: Editor
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: a2ca9860-e4c2-49bf-8030-451ecab4ede1
│      Assigned to: Content Planner
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Planner
│           Status: ✅ Completed
├── 📋 Task: 09be3ee7-1dd3-455f-a325-4f193ec8d026
│      Assigned to: Content Writer
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Writer
│           Status: ✅ Completed
└── 📋 Task: 3d3a4878-13b5-47d6-99c6-b4aa7a1a4ad0
       Assigned to: Editor
       Status: ✅ Completed
    └── 🤖 Agent: Editor
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 3d3a4878-13b5-47d6-99c6-b4aa7a1a4ad0                                                                     │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 0640041f-4635-42a9-b5d8-6f94547a869f                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

# Visualização formatada do resultado

In [32]:
Markdown(result.raw)

# Unraveling Artificial Intelligence: A Comprehensive Guide to the Latest Trends and Key Insights

## Introduction
Artificial Intelligence (AI) is reshaping how we interact with technology, enabling machines to learn, adapt, and execute tasks that previously relied on human intelligence. This blog post delves deep into AI, covering its definition, evolution over time, and its pivotal role across various industries. From streamlining automation processes to enhancing decision-making capabilities, AI has become an essential component in today's technological landscape.

The Latest Trends in Artificial Intelligence
The realm of AI is experiencing rapid progress, particularly in machine learning algorithms that power intelligent systems capable of improving performance continually without explicit programming. Additionally, the rise of autonomous vehicles and robotics demonstrates how AI is revolutionizing transportation and manufacturing sectors. In healthcare, AI is revolutionizing diagnostics, personalized treatments, and drug discovery, ushering in a new era of precision medicine. With these advancements, however, ethical concerns surrounding AI development, such as privacy, bias, and accountability, come to the forefront.

## Key Players in the AI Industry
Major tech titans like Google, Amazon, and Microsoft are investing significantly in AI research and development, leading groundbreaking projects across various domains. Simultaneously, AI startups are challenging traditional industries by leveraging innovative technologies to enhance efficiency and competitiveness. Academic institutions such as MIT and Stanford are driving AI research forward, fostering collaborations that push the boundaries of artificial intelligence and machine learning.

## Noteworthy News in Artificial Intelligence
Recent breakthroughs in natural language processing have unlocked new possibilities for conversational AI, enabling more natural interactions between machines and users. While AI promises increased productivity and innovation, concerns about its impact on the job market linger, sparking conversations on workforce reskilling and the future of work. Beyond traditional applications, AI is being utilized to tackle pressing global issues, including climate change mitigation through data-driven solutions. As AI technologies evolve, regulatory developments are shaping the governance framework for responsible AI deployment.

## Target Audience Analysis
Tech and IT professionals are seeking AI insights to stay competitive and drive digital transformation within their organizations. Business owners are exploring AI integration to streamline operations, enhance customer experiences, and uncover new revenue streams. Students and academics are actively engaged in AI research and development, contributing to the innovation landscape. For the broader audience, understanding AI's societal impact is crucial to navigate the increasingly AI-driven world we inhabit.

## SEO Keywords
Artificial Intelligence trends, Top AI companies, AI advances 2021, Ethics in AI, AI news updates

## Resources
Stay informed about the dynamic world of Artificial Intelligence by following our blog for regular updates on the latest trends, key players, and impactful news shaping the AI landscape. Utilize reports from leading AI research institutions, articles from reputable tech news outlets, whitepapers, case studies on AI implementation, and insights from AI experts and industry leaders to stay abreast in the evolving AI landscape.

In conclusion, this comprehensive blog post aims to unpack the multifaceted realm of Artificial Intelligence, offering a nuanced perspective on its trends, industry players, and impactful news. Whether you are a professional seeking cutting-edge insights, a business owner navigating digital transformation, an academic diving into research, or a curious individual interested in AI's societal implications, this guide is tailored to illuminate the dynamic world of AI technology and its profound implications.

# Rodando o Crew com o tema escolhido por mim

In [33]:
# Tópico: Economia recente da Argentina
result = crew.kickoff(inputs={"topic": "Recent Economy of Argentina"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 0640041f-4635-42a9-b5d8-6f94547a869f                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: a2ca9860-e4c2-49bf-8030-451ecab4ede1
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: a2ca9860-e4c2-49bf-8030-451ecab4ede1
       Status: Executing Task...
    └── 🤖 Agent: Content Planner
            Status: In Progress

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Recent Economy of Argentina.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


🚀 Crew: crew
└── 📋 Task: a2ca9860-e4c2-49bf-8030-451ecab4ede1
       Status: Executing Task...
    └── 🤖 Agent: Content Planner
            Status: In Progress
        └── 🧠 Thinking...

...



# Agent: Editor
## Final Answer: 
# Navigating the Recent Economy of Argentina: Trends, Players, and Insights for Informed Decisions

## Introduction
Argentina, renowned for its rich cultural heritage and varied landscapes, has recently encountered economic challenges that have garnered worldwide attention. The economic landscape of Argentina has been dynamic due to various factors, impacting both local businesses and international investors.

## Latest Trends in the Argentine Economy
### Inflation and Economic Growth
A significant trend in Argentina's economy is the struggle with high inflation rates, affecting the purchasing power of its citizens. Despite efforts to stabilize inflation, economic growth has been sluggish, raising concerns among investors about the country's financial stability.

### Foreign Exchange Market Fluctuations
The Argentine peso has experienced notable fluctuations in the foreign exchange market, influencing the competitiveness of Argentine exports and impo

🚀 Crew: crew
├── 📋 Task: a2ca9860-e4c2-49bf-8030-451ecab4ede1
│      Assigned to: Content Planner
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Planner
│           Status: ✅ Completed
├── 📋 Task: 09be3ee7-1dd3-455f-a325-4f193ec8d026
│      Assigned to: Content Writer
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Writer
│           Status: ✅ Completed
└── 📋 Task: 3d3a4878-13b5-47d6-99c6-b4aa7a1a4ad0
       Status: Executing Task...
    └── 🤖 Agent: Editor
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: a2ca9860-e4c2-49bf-8030-451ecab4ede1
│      Assigned to: Content Planner
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Planner
│           Status: ✅ Completed
├── 📋 Task: 09be3ee7-1dd3-455f-a325-4f193ec8d026
│      Assigned to: Content Writer
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Writer
│           Status: ✅ Completed
└── 📋 Task: 3d3a4878-13b5-47d6-99c6-b4aa7a1a4ad0
       Assigned to: Editor
       Status: ✅ Completed
    └── 🤖 Agent: Editor
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 3d3a4878-13b5-47d6-99c6-b4aa7a1a4ad0                                                                     │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 0640041f-4635-42a9-b5d8-6f94547a869f                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [34]:
# Possibilita a visualização de forma limpa
Markdown(result.raw)

# Navigating the Recent Economy of Argentina: Trends, Players, and Insights for Informed Decisions

## Introduction
Argentina, renowned for its rich cultural heritage and varied landscapes, has recently encountered economic challenges that have garnered worldwide attention. The economic landscape of Argentina has been dynamic due to various factors, impacting both local businesses and international investors.

## Latest Trends in the Argentine Economy
### Inflation and Economic Growth
A significant trend in Argentina's economy is the struggle with high inflation rates, affecting the purchasing power of its citizens. Despite efforts to stabilize inflation, economic growth has been sluggish, raising concerns among investors about the country's financial stability.

### Foreign Exchange Market Fluctuations
The Argentine peso has experienced notable fluctuations in the foreign exchange market, influencing the competitiveness of Argentine exports and imports. These fluctuations have introduced uncertainty for businesses within the country and have complicated future investment planning.

### Government Policies and Their Impact
Government policies play a pivotal role in shaping Argentina's economic landscape. Recent policy decisions have targeted issues such as inflation and trade imbalances, but the effectiveness and long-term consequences of these policies are topics of ongoing discussion among experts.

## Key Players Shaping Argentina's Economy
### Government Institutions
Government institutions, including the Central Bank of Argentina and the Ministry of Economy, are central in formulating and executing economic policies that steer the country's economic direction. The decisions made by these institutions have wide-ranging impacts on various sectors of the economy.

### Major Industries and Corporations
Vital sectors such as agriculture, manufacturing, and technology are the backbone of Argentina's economy. Major corporations operating within these sectors not only substantially contribute to the country's GDP but also possess the ability to influence policy-making and economic trends.

### International Organizations and Investors
International organizations like the IMF and foreign investors play a pivotal role in bolstering Argentina's economy through financial aid and investments. Their involvement can either stabilize or further disrupt the economic landscape of the country.

## Noteworthy News in the Argentine Economy
### Recent Developments in Trade Agreements
Argentina has been actively seeking trade agreements with various nations to enhance its exports and attract foreign investments. These agreements have the potential to unlock new markets for Argentine products and fortify its standing in the global economy.

### Impact of Global Economic Conditions on Argentina
The global economic scenario significantly impacts Argentina's economy, with fluctuations in commodity prices and trade relations affecting the country's export-dependent industries. Adapting to these global conditions is imperative for Argentina's economic resilience.

### Investment Trends and Opportunities
Despite economic hurdles, Argentina presents investment opportunities in sectors like renewable energy, technology, and infrastructure. Understanding market trends and the regulatory framework is crucial for investors interested in capitalizing on these opportunities.

## Target Audience Analysis
This blog post is tailored for business professionals, investors, and economists seeking insights into Argentina's economy. By delving into the latest trends and key players shaping the Argentine economy, readers can make informed decisions regarding their investments and business strategies in the country.

## Interest and Pain Points of the Audience
Audience members are keen on understanding market trends to seize opportunities in Argentina while being cognizant of economic instability and policy uncertainties that could impact their financial choices. Offering insights into navigating these challenges is essential to meet the audience's requirements.

## SEO Keywords
Argentina economy, economic trends Argentina, Argentine industries, investment opportunities Argentina

Stay updated on Argentina's economy by following our blog for regular updates, analysis, and insights sourced from reliable outlets such as the Central Bank of Argentina reports and analysis from esteemed financial news sources. Additionally, reports from international organizations like the IMF provide a broader perspective on Argentina's economic landscape, offering insights into challenges and opportunities.

**Call to Action:** Stay informed about Argentina's economy by following our blog for regular updates, analysis, and insights.

This blog post aims to empower readers with the knowledge and understanding needed to navigate the intricacies of the Argentine economic landscape confidently and insightfully through a comprehensive analysis of the recent economy of Argentina.